[Notion](https://www.notion.so/aspirina180mg/Soy-Henry-49afb64d6c0641cc80dbc0378dd2db34?pvs=4)

[Kick-Off](https://drive.google.com/file/d/1l1SM-UAw5Mdkr6lmQ9X5UI4cxU_FRg_D/view)

[Slides](https://docs.google.com/presentation/d/1kRrCZDL-ZgZnm4DQ62Ovelf92uP-s4b7cdtp3gkK_kU/edit#slide=id.g2420ab8e36f_0_208)

[Repositorio](https://github.com/soyHenry/PI_DA/tree/Full_Time)

[Rúbrica](https://docs.google.com/spreadsheets/d/e/2PACX-1vTy9Ome3iLIl40SZ2jciZLiwARr9MVXfo_Mud3vVCin9P4zroAw_oxVQVch6m4TKivmUjFrYJKMJJhS/pubhtml)


# **ETL**


Este Archivo contiene el proceso de ETL, de los datos encontrados en los archivos `homicidios.xlsx` y `lesiones.xlsx`, disponibles en la carpeta `Datasets` de este mismo repositorio.
Durante el proceso de ETL se realizarán las siguientes aciones:

- Eliminarn columnas redundantes o duplicads.
- Reemplazar datos faltantes.
- Eliminar columnas con falta de información necesaria.
- Transforma los tipos de datos a tipos más adecuados
- Exportar a formato parquet.


## **Importación de librerías**


- `numpy` para realisar cálculos matemáticos y estadísticos.
- `pandas` para realizar análisis de datos.


In [1]:
import numpy as np
import pandas as pd

## **Carga de datos y creación de dataframes**


### **Se carga cada hoja del documento `homicidios.xlsx` a dataframes por separado.**


- HECHOS se carga en df_hechos
- VICTIMAS se carga en df_victimas


In [3]:
df_hechos = pd.read_excel("Datasets\homicidios.xlsx", sheet_name="HECHOS")
df_victimas = pd.read_excel("Datasets\homicidios.xlsx", sheet_name="VICTIMAS")

## **Se eliminan columnas redundantes o duplicadas entre `df_hechos` y `df_victimas`**


columnas eliminadas de df_hechos

- AAAA (información se encuentra en `df_hechos.FECHA`).
- MM (información se encuentra en `df_hechos.FECHA`).
- DD (información se encuentra en `df_hechos.FECHA`).
- HH (información se encuentra en `df_hechos.HORA`).
- df_hechos.XY (CABA) (información se encuentra en `pos x` y `pos y`).
- pos y (información se encuentra en `df_hechos.XY (CABA)`).
- PARTICIPANTES (información se encuentra en `df_hechos.ACUSADO` y en `df_victimas.VICTIMA`).
- VICTIMA (cuplicado de `df_victimas.VICTIMA`).
  <br><br>
  columnas eliminadas de df_victimas
- FECHA (duplicado de `df_hechos.FECHA`).
- AAAA (información se encuentra en `df_hechos.FECHA`).
- MM (información se encuentra en `df_hechos.FECHA`).
- DD (información se encuentra en `df_hechos.FECHA`).


In [5]:
df_hechos = df_hechos.drop(
    ["AAAA", "MM", "DD", "HH", "XY (CABA)", "PARTICIPANTES", "VICTIMA"], axis=1
)
df_victimas = df_victimas.drop(["FECHA", "AAAA", "MM", "DD"], axis=1)

## **Se crea el dataframe `df_homicidios` mezclando df_hechos y df_victimas a través de `df_hechos.ID` y `df_victimas.ID_Hecho`**


In [7]:
df_homicidios = pd.merge(df_hechos, df_victimas, left_on="ID", right_on="ID_hecho")
df_homicidios = df_homicidios[
    [
        "ID",
        "ACUSADO",
        "FECHA",
        "HORA",
        "COMUNA",
        "LUGAR_DEL_HECHO",
        "TIPO_DE_CALLE",
        "Calle",
        "Altura",
        "Cruce",
        "Dirección Normalizada",
        "pos x",
        "pos y",
        "N_VICTIMAS",
        "VICTIMA",
        "ROL",
        "SEXO",
        "EDAD",
        "FECHA_FALLECIMIENTO",
    ]
]
del df_hechos
del df_victimas

## **Se reemplazan los datos con "SD" (Sin dato) por NaN**


In [9]:
pd.set_option("future.no_silent_downcasting", True)

df_homicidios = df_homicidios.infer_objects(copy=False)
df_homicidios = df_homicidios.replace("SD", np.nan)
df_homicidios = df_homicidios.replace("sd", np.nan)

- Filas con datos faltantes en "ACUSADO" y "VICTIMA" al mismo tiempo.
- Filas con datos faltantes en "HORA".
- Filas con datos faltantes en "VICTIMA" y "ROL" al mismo tiempo.
- Filas con datos faltantes en "LUGAR_DEL_HECHO", "Dirección Normalizada" y "Calle" al mismo tiempo.


In [11]:
columnas_revisadas = ["ACUSADO", "VICTIMA"]
filas_invalidas = df_homicidios[
    df_homicidios[columnas_revisadas].isnull().all(axis=1)
].index
df_homicidios = df_homicidios.drop(filas_invalidas)

columnas_revisadas = ["HORA"]
filas_invalidas = df_homicidios[
    df_homicidios[columnas_revisadas].isnull().all(axis=1)
].index
df_homicidios = df_homicidios.drop(filas_invalidas)

columnas_revisadas = ["VICTIMA", "ROL"]
filas_invalidas = df_homicidios[
    df_homicidios[columnas_revisadas].isnull().all(axis=1)
].index
df_homicidios = df_homicidios.drop(filas_invalidas)

columnas_revisadas = ["LUGAR_DEL_HECHO", "Dirección Normalizada", "Calle"]
filas_invalidas = df_homicidios[
    df_homicidios[columnas_revisadas].isnull().all(axis=1)
].index
df_homicidios = df_homicidios.drop(filas_invalidas)

del columnas_revisadas
del filas_invalidas

## **Revisión de tipo de datos por columna y conversión de tipo de datos**


- Convertir la columna "FECHA" a tipo datetime
- Convertir la columna "HORA" a tipo datetime
- Convertir la columna "COMUNA" a tipo integer de 8 bits
- Convertir la columna "Altura" a tipo flotante de 32 bits
- Convertir la columna "pos x" a tipo flotante de 32 bits
- Convertir la columna "pos y" a tipo flotante de 32 bits
- Convertir la columna "N_VICTIMAS" a tipo integer de 8 bits
- Convertir la columna "EDAD" a tipo integer de 8 bits
- Convertir la columna "FECHA_FALLECIMIENTO" a tipo datetime


In [13]:
df_homicidios["FECHA"] = pd.to_datetime(df_homicidios["FECHA"], format="%Y-%m-%d")
df_homicidios["HORA"] = pd.to_datetime(df_homicidios["HORA"], format="%H:%M:%S")
df_homicidios["COMUNA"] = df_homicidios["COMUNA"].astype("int8")
df_homicidios["Altura"] = df_homicidios["Altura"].astype("float32")
df_homicidios["pos x"] = df_homicidios["pos x"].replace(".", np.nan).astype("float32")
df_homicidios["pos y"] = df_homicidios["pos y"].replace(".", np.nan).astype("float32")
df_homicidios["N_VICTIMAS"] = df_homicidios["N_VICTIMAS"].astype("int8")
df_homicidios["EDAD"] = df_homicidios["EDAD"].fillna(0)
df_homicidios["EDAD"] = df_homicidios["EDAD"].astype("int8")
df_homicidios["FECHA_FALLECIMIENTO"] = pd.to_datetime(
    df_homicidios["FECHA_FALLECIMIENTO"]
)

# Exportación a archivo Parquet


In [15]:
df_homicidios.to_parquet("Datasets/df_homicidios.parquet")
del df_homicidios